In [1]:
import syft as sy

In [2]:
duet2 = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!

`searchable` is deprecated please use `pointable` in futurees: 1  Request Handlers: 0                                


In [3]:
import numpy as np
import torch as th
import pandas as pd
from PIL import Image
import imageio.v2 as imageio
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
df =pd.read_csv('HAM10000_metadata.csv')

In [5]:
df['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [6]:
target = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [7]:
subset_metadata_a = df.sample(frac=0.2, random_state=42) 

In [8]:
subset_metadata_a

,lesion_id,image_id,dx,dx_type,age,sex,localization
1617,HAM_0007180,ISIC_0033272,mel,histo,65.0,male,face
8128,HAM_0007195,ISIC_0031923,nv,histo,40.0,female,lower extremity
2168,HAM_0001835,ISIC_0026652,mel,histo,65.0,male,back
1090,HAM_0000465,ISIC_0030583,bkl,consensus,35.0,female,trunk
7754,HAM_0001720,ISIC_0034010,nv,histo,45.0,male,abdomen
...,...,...,...,...,...,...,...
1770,HAM_0000375,ISIC_0030238,mel,histo,45.0,male,back
1888,HAM_0006376,ISIC_0034317,mel,histo,70.0,female,lower extremity
2020,HAM_0004979,ISIC_0032245,mel,histo,85.0,male,back
1642,HAM_0005711,ISIC_0033779,mel,histo,35.0,male,back


In [9]:
num_samples_per_label_a = 400                                                 ## 200 sample for each class
balanced_sub_a = pd.DataFrame()
for label in target:
    label_metadata_a = subset_metadata_a[subset_metadata_a['dx'] == label]
    label_samples_a = label_metadata_a.sample(n=num_samples_per_label_a, random_state=42, replace = True)
    balanced_sub_a = pd.concat([balanced_sub_a, label_samples_a])

In [10]:
balanced_sub_a

,lesion_id,image_id,dx,dx_type,age,sex,localization
9707,HAM_0005931,ISIC_0031191,akiec,histo,60.0,male,lower extremity
9729,HAM_0007601,ISIC_0032437,akiec,histo,65.0,male,upper extremity
9924,HAM_0006434,ISIC_0030142,akiec,histo,60.0,male,lower extremity
9936,HAM_0006887,ISIC_0029500,akiec,histo,45.0,male,face
9888,HAM_0005231,ISIC_0025247,akiec,histo,70.0,male,lower extremity
...,...,...,...,...,...,...,...
2344,HAM_0004455,ISIC_0027385,vasc,histo,20.0,male,trunk
2453,HAM_0001038,ISIC_0024706,vasc,consensus,50.0,male,trunk
2406,HAM_0003480,ISIC_0032890,vasc,consensus,70.0,female,lower extremity
2392,HAM_0006748,ISIC_0027210,vasc,consensus,30.0,female,lower extremity


In [11]:
sub_a_class_count = balanced_sub_a['dx'].value_counts()

In [12]:
size = (64, 64)
images_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_a = imageio.imread('HAM_data//HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_a).resize(size))
    images_sub_a.append(resized_img)

In [13]:
images_and_labels_sub_a = []
for i, row in balanced_sub_a.iterrows():
    img_b = imageio.imread('HAM_data/HAM10000/' + row['image_id'] + '.jpg')
    resized_img = np.array(Image.fromarray(img_b).resize(size))
    label = row['dx']
    images_and_labels_sub_a.append((resized_img, label))

In [14]:
images_sub_a, labels_sub_a = zip(*images_and_labels_sub_a)
images_sub_a = np.array(images_sub_a)
labels_sub_a = np.array(labels_sub_a)

# Print the shapes of the resulting arrays
print('Image shape:', images_sub_a.shape)
print('Label shape:', labels_sub_a.shape)

Image shape: (2800, 64, 64, 3)
Label shape: (2800,)


In [15]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_sub_a)

In [16]:
image_tensor = th.FloatTensor(images_sub_a.astype('float32'))
label_tensor = th.FloatTensor(labels_encoded.astype('float32'))

In [17]:
import torchvision.transforms as transforms
from PIL import Image
import torch

transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=0),    
    transforms.RandomHorizontalFlip(),    
    transforms.ToTensor(),    
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Apply the transformations to the data
x_train_transformed = []
for img in images_sub_a:
    img_pil = Image.fromarray(img)
    img_transformed = transform_train(img_pil)
    x_train_transformed.append(img_transformed)

In [18]:
x_train_transformed = torch.stack(x_train_transformed)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x_train_transformed, label_tensor, train_size=0.7, random_state=2019)

In [20]:
x_train = x_train.tag("worker2 (x_train)")
y_train = y_train.tag("worker2 (y_train)")
x_test = x_test.tag("worker2 (x_test)")
y_test = y_test.tag("worker2 (y_test)")

In [21]:
x_train_image = x_train.send(duet2, searchable = True)
x_test_image = x_test.send(duet2, searchable = True)
y_train_label = y_train.send(duet2, searchable = True)
y_test_lable = y_test.send(duet2, searchable = True)

`searchable` is deprecated please use `pointable` in future
`searchable` is deprecated please use `pointable` in future
`searchable` is deprecated please use `pointable` in future


In [22]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: a900d928059e400bb5ce0f1a5e9e94a2>,[worker2 (x_train)],,<class 'torch.Tensor'>
1,<UID: 2e36cefae90347ff98240428d9f27279>,[worker2 (x_test)],,<class 'torch.Tensor'>
2,<UID: ccadcf7028474e179308596704008f2c>,[worker2 (y_train)],,<class 'torch.Tensor'>
3,<UID: 409e7313ff48483da8d36516f39a6d84>,[worker2 (y_test)],,<class 'torch.Tensor'>


In [23]:
duet2.requests.add_handler(action="accept")